In [27]:
from tkinter import *
from tkinter import ttk, messagebox
from tkinter.filedialog import askopenfilenames, askopenfilename

In [28]:
#################### constants ###############################

constants = {
    'width':50,
    'height':50,
    'font':('Times 12 bold'),
    'bg':'white',
    'bd':5,
    'anchor':'w',
    'relief':SOLID,
}
##############################################################
##############################################################

#################### Variables ###############################

felix_files_type = ("Felix Files", "*.felix")
mass_files_type = ("Mass Files", "*.mass")
time_files_type = ("Timescan Files", "*.scan")
pow_files_type = ("Pow Files", "*.pow")
all_files_type = ("All files", "*")
##############################################################
##############################################################

#################### Definitions #############################

def var_check(kw):
    for i in constants:
        if not i in list(kw.keys()):
            kw[i] = constants[i]
    return kw

##############################################################
##############################################################

In [29]:
class FELion(Tk):

    def __init__(self, *args, **kwargs):
        
        Tk.__init__(self, *args, **kwargs)
        Tk.wm_geometry(self, '400x400')
        container = Frame(self)

        container.pack(side="top", fill="both", expand = True)

        container.grid_rowconfigure(0, weight=1)
        container.grid_columnconfigure(0, weight=1)
        
        self.frames = {}

        for F in (StartPage,):

            frame = F(container, self)

            self.frames[F] = frame

            frame.grid(row=0, column=0, sticky="nsew")

        self.show_frame(StartPage)

    def show_frame(self, cont):

        frame = self.frames[cont]
        frame.tkraise()

In [26]:
class Entry_widgets(Frame):
    
    def __init__(self, parent, method,  *args, **kw):
        Frame.__init__(self, parent)
        self.parent = parent
        self.txt, x, y = args[0], args[1], args[2]
        kw = var_check(kw)
        
        if method == 'Entry':
            if isinstance(self.txt, str): self.value = StringVar()
            elif isinstance(self.txt, int): self.value = IntVar()
                
            self.value.set(self.txt)
            self.entry = Entry(self.parent, bg = kw['bg'], bd = kw['bd'], textvariable = self.value, font = kw['font'])
            self.entry.place(relx = x, rely = y, anchor = kw['anchor'], width = kw['width'], height = kw['height'])
                
        elif method == 'Check':
            self.value = BooleanVar()
            if 'default' in kw: self.value.set(kw['default'])
            else: self.value.set(False)
                
            Check = ttk.Checkbutton(self.parent, text = self.txt, variable = self.value)
            Check.place(relx = x, rely = y, width = kw['width'], height = kw['height'])
     
    def get(self):
        return self.value.get()

In [43]:
class FELion_widgets(Frame):
    
    def __init__(self, parent):
        Frame.__init__(self, parent)

        self.parent = parent
        self.location = "/"
        self.fname = ""
        self.filelist = []
              
    def labels(self, *args, **kw):
        self.txt = args[0]
        x, y = args[1], args[2]
        kw = var_check(kw)
        
        self.label = Label(self.parent, text = self.txt, font = kw['font'], bg = kw['bg'], bd = kw['bd'], relief = kw['relief'])
        self.label.place(relx = x, rely = y, anchor = kw['anchor'], width = kw['width'], height = kw['height'])
    
    def buttons(self, *args, **kw):
        btn_txt = args[0]
        x, y = args[1], args[2]
        func = args[3]
        kw = var_check(kw)
        
        self.button = ttk.Button(self.parent, text = btn_txt, command = lambda: func())
        self.button.place(relx = x, rely = y, width = kw['width'], height = kw['height'])

    def open_dir(self, file_type):
        root = Tk()
        root.withdraw()

        root.filename =  askopenfilename(initialdir = self.location, title = "Select file", filetypes = (file_type, ("all files","*.*")))
        filename = root.filename
        filename = filename.split("/")

        self.fname = filename[-1]
        del filename[-1]
        
        self.location = "/".join(filename)
        root.destroy()
        
        return self.fname, self.location

    def openfilelist(self, file_type):
        self.filelist = [] # to prevent appending previously selected files
        self.openlist = askopenfilenames(initialdir=self.location, initialfile='tmp',
                        filetypes=[file_type, ("All files", "*")])

        for i in self.openlist:
            location = i.split("/")
            file = location[-1]
            self.filelist.append(file)
            del location[-1]
            self.location = "/".join(location)

        return self.filelist, self.location
    
    def entries(self, method,  *args, **kw):
        txt, x, y = args[0], args[1], args[2]
        kw = var_check(kw)
        
        if method == 'Entry':
            if isinstance(txt, str): self.parent.txt = StringVar()
            elif isinstance(txt, int): self.parent.txt = IntVar()
                
            self.parent.txt.set(txt)
            self.parent.entry = Entry(self.parent, bg = kw['bg'], bd = kw['bd'], textvariable = self.parent.txt, font = kw['font'])
            self.parent.entry.place(relx = x, rely = y, anchor = kw['anchor'], width = kw['width'], height = kw['height'])
            
            return self.parent.txt


In [44]:
class StartPage(Frame):
    def __init__(self, parent, controller):
        Frame.__init__(self, parent)
        self.widget = FELion_widgets(self)
        
        # Creating labels
        self.widget.labels('First Name: ', 0.1, 0.1, font = 'Times 12 italic', width = 100, bd = 1, relief = SOLID)
        self.widget.labels('Last Name: ', 0.1, 0.2, font = 'Times 12 italic', width = 100)

        #Creating Entry widgets
        self.firstName = self.widget.entries('Entry', 'John ', 0.4, 0.1, height = 25, width = 100)
        self.lastName = self.widget.entries('Entry', 'John', 0.4, 0.2, height = 25, width = 100)
        
        #Creating buttons
        self.widget.buttons('Print', 0.2, 0.5, self.test_func1)
        self.widget.buttons('Open', 0.4, 0.5, self.filelists)
        
        # Check_button widget
        #self.check1 = Entry_widgets(self,'Check', 'plot', 0.1, 0.7, default = True)
        
    def test_func1(self):
        test_func(self.firstName.get(), self.lastName.get())
    
    def filename_location(self):
        self.fname, self.location = self.widget.open_dir(pow_files_type)
        print('Filename: %s \nLocation: %s \n'%(self.fname, self.location))
    
    def filelists(self): 
        self.filelists, self.location = self.widget.openfilelist(all_files_type)
        print('Filelists: {}, \nLocation: {}'.format(self.filelists, self.location))
        
            
root = FELion()

root.mainloop()

PY_VAR11
PY_VAR12


In [32]:
def test_func(f, l):
    print('Hey ', f + l)

0.591351144


In [1]:
from tkinter import *
def donothing():
   filewin = Toplevel(root)
   button = Button(filewin, text="Do nothing button")
   button.pack()
   
root = Tk()
menubar = Menu(root)
filemenu = Menu(menubar, tearoff = 0)
filemenu.add_command(label="New", command = donothing)
filemenu.add_command(label = "Open", command = donothing)
filemenu.add_command(label = "Save", command = donothing)
filemenu.add_command(label = "Save as...", command = donothing)
filemenu.add_command(label = "Close", command = donothing)

filemenu.add_separator()

filemenu.add_command(label = "Exit", command = root.quit)
menubar.add_cascade(label = "File", menu = filemenu)
editmenu = Menu(menubar, tearoff=0)
editmenu.add_command(label = "Undo", command = donothing)

editmenu.add_separator()

editmenu.add_command(label = "Cut", command = donothing)
editmenu.add_command(label = "Copy", command = donothing)
editmenu.add_command(label = "Paste", command = donothing)
editmenu.add_command(label = "Delete", command = donothing)
editmenu.add_command(label = "Select All", command = donothing)

menubar.add_cascade(label = "Edit", menu = editmenu)
helpmenu = Menu(menubar, tearoff=0)
helpmenu.add_command(label = "Help Index", command = donothing)
helpmenu.add_command(label = "About...", command = donothing)
menubar.add_cascade(label = "Help", menu = helpmenu)

root.config(menu = menubar)
root.mainloop()

In [22]:
class MyClass:
    
    def __init__(self, x):
        self.x = x
    
    def method(self):
        return 'instance method called', self.x

    @classmethod
    def classmethod(cls):
        return 'class method called', cls

    @staticmethod
    def staticmethod():
        return 'static method called'
    
    def __repr__(self):
        return f'Class: {self.x!r}'
    
    def __str__(self):
        return f'MyClass()'

In [24]:
obj = MyClass(12)
f'{obj}'

'MyClass()'

In [25]:
firstName = 'John'
lastName = 'Dave'
age = 12
f'Hey {firstName}, is your lastname {lastName} and I heard you are {age} old. Is it True?'

'Hey John, is your lastname Dave and I heard you are 12 old. Is it True?'

In [57]:
import os
from os.path import join
location = 'D:\\FELion-Spectrum-Analyser\\testing'
name = 'Mass.png'
join(location, f'{name}')

'D:\\FELion-Spectrum-Analyser\\testing\\Mass.png'

In [73]:
import glob2
glob2.glob('*.png')
f = 'power.png'
f.find('p')

0